In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
using NonlinearSolve
import DifferentialEquations as DE
include("multiharmonic_balance.jl");
using Random

Equation parameters

In [ ]:
gamma = 0.25;
gamma3 = 0.0;
g0::Float64 = 3; # m / s^2
height = 3; # m
Nx = 100;
harmonics = 4; # number of harmonics


Discretization parameters

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
stepx = (xright-xleft)/Nx;

**Declare the problem**

In [ ]:
# Define symbolics
Model.@parameters x, t;

const Dx = Model.Differential(x);
const Dt = Model.Differential(t);

In [ ]:
for gino in 2:2:10
    print(gino)
end

In [ ]:
u0 = rand((Nx+1) * harmonics * 2)*.4;
# u0[1] = 0.3;

for gamma3 in 0:0.005:0.8
    
    omega = omega * 1.0;
    
    vars, var_exprs, (u,) = create_ansatz((x,), t, omega, harmonics);
    
    F = 100*exp(-40*x^2)* sin(omega*t);
    
    pde::Symbolics.Num = Dt(Dt(u)) - g0*height*(Dx(Dx(u))) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u);
    
    expanded = expand_trig_jl(pde, t, omega)
    eqns = make_residual(expanded, harmonics, omega, t)
    sym_eqs = map(transform_sym(Nx,) ∘ Meta.parse ∘ string, eqns)
    
    resid = create_residual_function(sym_eqs, vars, Nx, 0,
        Dict(
        :A => [:(0.3), nothing],
        :B => [:(0.0), nothing],
        :C => [:(0.0), nothing],
        :D => [:(0.0), nothing],
        :E => [:(0.0), nothing],
        :F => [:(0.0), nothing],
        :G => [:(0.0), nothing],
        :H => [:(0.0), nothing]));
    
    residual! = eval(resid)
    N = (Nx+1)
    
    f = NonlinearFunction(residual!)
    prob = NonlinearProblem(f, u0, [stepx])
    sol = solve(prob, NewtonRaphson(), reltol=1e-5, abstol=1e-5, maxiters=1000)

    u0 = sol.u
    println(gamma3, " ", sol.retcode)
end

In [ ]:
solutions = [reshape(sol.u[(k-1)*N+1:k*N], Nx+1) for k in 1:(2*harmonics)];

In [ ]:
using Plots

xgrid = collect(range(start=0.0, stop=1.0, step=stepx))

anim = @animate for t in 1:0.01:10
    if t % 100 == 0
        println(t)
    end
    u_new = zeros(length(xgrid))
        j = 1
        for i in 1:(2*harmonics)
            if isodd(i)
                u_new .+= solutions[i] .* sin(j * omega * t)
            else
                u_new .+= solutions[i] .* cos(j * omega * t)
                j += 1
            end
        end
    plot(u_new)
    title!("Wave Equation with H = $(harmonics), \$\\gamma\$ = $(gamma), \$\\gamma_3\$ = $(gamma3), \$\\omega\$ = $omega", titlefontsize=10)
    ylims!(-1, 1)
end

gif(anim, "HB_WE_1D.gif", fps=60)